In [3]:
from pandas import read_csv
from os import path

In [4]:
dataframe = read_csv(path.join("..", 'data/raw','carData.csv'))
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [5]:
dataframe.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


# Dataset's informations
We have a dataset of **301 observations** with **9 features** about cars. Each observation is information about a specific car model.  
We have 5 **numericals** features and 4 **qualitatives** features (3 are **categoricals**)

In [6]:
dataframe.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.627907,4.661296,7.628472,36947.205980,0.043189
std,2.891554,5.082812,8.644115,38886.883882,0.247915
min,2003.000000,0.100000,0.320000,500.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,0.000000
75%,2016.000000,6.000000,9.900000,48767.000000,0.000000
max,2018.000000,35.000000,92.600000,500000.000000,3.000000


- compare selling & present price  
- histogram of year & owner 

# Describe quantitatives data  
The dataset is about the 301 cars models between 2003 & 2018. 50% of them were created between 2003 & 2014, the other half between 2014 & 2018.  
The selling price start at 0.1 and finish at 35, with the median of the prices at 3.6.  
In opposition, the present price is seems to be generally bigger, because it start at 0.3 and finish at 92 with the median of the prices at 6.4.  
The kms driven go from 500 km to 500 000 km with half more than 32 000 km.  
75% of the cars have 0 owner and it can go to 3 owners maximum.